<a href="https://colab.research.google.com/github/jzsyuan/TaipowerTutorial/blob/main/Codis_Tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 網路爬蟲

氣溫網址: http://e-service.cwb.gov.tw/HistoryDataQuery/QueryDataController.do?command=viewMain

工具: 
1. URLencode and URLdecode: https://tool.chinaz.com/tools/urlencode.aspx

In [ ]:
import requests    # 爬蟲
import pandas as pd # 資料處理
import re       # regex 文字正規化   
import urllib     # 爬蟲及處理url字串轉ASCII
import time      # time sleep
import logging  # log
FORMAT = '%(asctime)s | %(thread)d |: [%(levelname)s]: %(message)s' # log format time : thread id | Info/success/error | message
logging.basicConfig(level=logging.INFO,format = FORMAT)


# 先看如何得到資料
 

*   台北市鞍部2021-12-01: http://e-service.cwb.gov.tw/HistoryDataQuery/DayDataController.do?command=viewMain&station=466910&stname=%25E9%259E%258D%25E9%2583%25A8&datepicker=2021-12-01


In [ ]:
url = 'http://e-service.cwb.gov.tw/HistoryDataQuery/QueryDataController.do?command=viewMain'
r = requests.get(url,verify=False)

In [ ]:
print(r)
print(r.text)

In [ ]:
r.encoding = 'utf-8'
vlistStr = r.text.split("stList = {")[1].split("areaList = {")[0].replace("\r\n","").replace("\t","").replace(" ","").split("};")[0]
print(vlistStr)

"466930":["竹子湖","ZHUZIHU","臺北市","1"],"466940":["基隆","KEELUNG","基隆市","1"],"466950":["彭佳嶼","PENGJIAYU","基隆市","1"],"466990":["花蓮","HUALIEN","花蓮縣","1"],"467050":["新屋","XINWU","桃園市","1"],"467060":["蘇澳","SU-AO","宜蘭縣","1"],"467080":["宜蘭","YILAN","宜蘭縣","1"],"467110":["金門","KINMEN","金門縣","1"],"467270":["田中","TianZhong","彰化縣","1"],"467300":["東吉島","DONGJIDAO","澎湖縣","1"],"467350":["澎湖","PENGHU","澎湖縣","1"],"467410":["臺南","TAINAN","臺南市","1"],"467420":["永康","YONGKANG","臺南市","1"],"467440":["高雄","KAOHSIUNG","高雄市","1"],"467480":["嘉義","CHIAYI","嘉義市","1"],"467490":["臺中","TAICHUNG","臺中市","1"],"467530":["阿里山","ALISHAN","嘉義縣","1"],"467540":["大武","DAWU","臺東縣","1"],"467550":["玉山","YUSHAN","南投縣","1"],"467571":["新竹","HSINCHU","新竹縣","1"],"467590":["恆春","HENGCHUN","屏東縣","1"],"467610":["成功","CHENGGONG","臺東縣","1"],"467620":["蘭嶼","LANYU","臺東縣","1"],"467650":["日月潭","SUNMOONLAKE","南投縣","1"],"467660":["臺東","TAITUNG","臺東縣","1"],"467770":["梧棲","WUQI","臺中市","1"],"467990":["馬祖","MATSU","連江縣","1"],"C0A520":["山佳","Shanjia","新

In [ ]:
# 使用regex 取出 站號，站名，縣市英文，縣市中文，該站的形態
regex = re.compile('"([0-9a-zA-Z]+?)"\:\["(\w+?)","(\w+?)","(\w+?)","(\w+?)"')

match = regex.findall(vlistStr)
match = [x for x in match if x[4]=="1"]
print(match)

[('466930', '竹子湖', 'ZHUZIHU', '臺北市', '1'), ('466940', '基隆', 'KEELUNG', '基隆市', '1'), ('466950', '彭佳嶼', 'PENGJIAYU', '基隆市', '1'), ('466990', '花蓮', 'HUALIEN', '花蓮縣', '1'), ('467050', '新屋', 'XINWU', '桃園市', '1'), ('467080', '宜蘭', 'YILAN', '宜蘭縣', '1'), ('467110', '金門', 'KINMEN', '金門縣', '1'), ('467270', '田中', 'TianZhong', '彰化縣', '1'), ('467300', '東吉島', 'DONGJIDAO', '澎湖縣', '1'), ('467350', '澎湖', 'PENGHU', '澎湖縣', '1'), ('467410', '臺南', 'TAINAN', '臺南市', '1'), ('467420', '永康', 'YONGKANG', '臺南市', '1'), ('467440', '高雄', 'KAOHSIUNG', '高雄市', '1'), ('467480', '嘉義', 'CHIAYI', '嘉義市', '1'), ('467490', '臺中', 'TAICHUNG', '臺中市', '1'), ('467530', '阿里山', 'ALISHAN', '嘉義縣', '1'), ('467540', '大武', 'DAWU', '臺東縣', '1'), ('467550', '玉山', 'YUSHAN', '南投縣', '1'), ('467571', '新竹', 'HSINCHU', '新竹縣', '1'), ('467590', '恆春', 'HENGCHUN', '屏東縣', '1'), ('467610', '成功', 'CHENGGONG', '臺東縣', '1'), ('467620', '蘭嶼', 'LANYU', '臺東縣', '1'), ('467650', '日月潭', 'SUNMOONLAKE', '南投縣', '1'), ('467660', '臺東', 'TAITUNG', '臺東縣', '1'), ('46777

In [ ]:
code,cname,name,city,status = match[0][0],match[0][1],match[0][2],match[0][3],match[0][4]
datepicker = '2021-12-07'
headers={'User-agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.61 Safari/537.36'}

In [ ]:
url = 'http://e-service.cwb.gov.tw/HistoryDataQuery/DayDataController.do'
f = {'command': 'viewMain', 'station': code, 'stname': urllib.parse.urlencode({"a": cname})[2:],
      'datepicker': datepicker}
url = url + "?" + urllib.parse.urlencode(f)
# 萃取id = MyTable 
df=pd.read_html(requests.get(url,headers=headers).text,attrs={'id':'MyTable'},encoding='UTF-8')[0]


2021-12-15 02:14:02,322 | 139657058264960 |: [INFO]: NumExpr defaulting to 2 threads.


In [ ]:
def Codis(datepicker,Match):
  """

  :param datepicker: "%Y-%m-%d"
  :param Match: list of tuple [(code,cname,name,city,status),(code,cname,name,city,status),...] 
  :return dataframe codis
  """

  headers={'User-agent':'Mozilla/5.0 (Windows NT 6.3; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.61 Safari/537.36'}
  for index,(code,cname,name,city,status) in enumerate(Match):
    url = 'http://e-service.cwb.gov.tw/HistoryDataQuery/DayDataController.do'
    f = {
        'command': 'viewMain', 
        'station': code, 
        'stname': urllib.parse.urlencode({"a": cname})[2:],
        'datepicker': datepicker
      }
    url = url + "?" + urllib.parse.urlencode(f)
    resp =requests.get(url,headers=headers,verify=False)

    if resp.status_code == 200:
      logging.info(f'{resp.status_code}:|{index}/{len(Match)}|{datepicker}-{city}-{cname}')
      df=pd.read_html(resp.text,attrs={'id':'MyTable'},encoding='UTF-8')[0]
    else:
      logging.error(f'{resp.status_code}:|{index}/{len(Match)}|{datepicker}-{city}-{cname}')
      continue
    

    # columns name 
    # 確認columns 是否為tuple  
    if isinstance(df.columns[0], tuple):
      # 一般狀況
      df.columns = [i[2].strip().replace(" ", "_") for i in df]
    else:
      # 曾經的例外狀況
      df = df[2:]
      df.columns = [i.strip().replace(" ", "_") for i in df.loc[2]]
      df = df[1:]
      df.index = range(1, len(df) + 1)

    # 插入測站代號、測站名、測站中文名、城市、日期
    df.insert(loc=0, column='StationCode', value=code)
    df.insert(loc=1, column='StationName', value=name)
    df.insert(loc=2, column='StationCName', value=cname)
    df.insert(loc=3, column='City', value=city)
    df.insert(loc=4, column='Date', value=datepicker)
    # 看有沒有dataSet 沒有的話去generate dataframe 欄位為
    try:
      len(dataSet)
    except:
      dataSet = pd.DataFrame(columns=df.columns)
    
    dataSet = pd.concat([dataSet, df], axis=0, ignore_index=True)

  dataSet = dataSet.fillna("NA")
  dataSet = dataSet.replace(("-", "", "...", "X"), "NA")
  dataSet.to_csv(f'CODIS_{datepicker}.csv', index=False)
  
  return dataSet


  

In [ ]:
Codis('2021-12-08',Match=match[0:10])

2021-12-15 02:14:02,829 | 139657058264960 |: [INFO]: 200:|0/10|2021-12-08-臺北市-竹子湖
2021-12-15 02:14:03,260 | 139657058264960 |: [INFO]: 200:|1/10|2021-12-08-基隆市-基隆
2021-12-15 02:14:04,135 | 139657058264960 |: [INFO]: 200:|2/10|2021-12-08-基隆市-彭佳嶼
2021-12-15 02:14:04,577 | 139657058264960 |: [INFO]: 200:|3/10|2021-12-08-花蓮縣-花蓮
2021-12-15 02:14:05,003 | 139657058264960 |: [INFO]: 200:|4/10|2021-12-08-桃園市-新屋
2021-12-15 02:14:05,897 | 139657058264960 |: [INFO]: 200:|5/10|2021-12-08-宜蘭縣-宜蘭
2021-12-15 02:14:06,325 | 139657058264960 |: [INFO]: 200:|6/10|2021-12-08-金門縣-金門
2021-12-15 02:14:06,751 | 139657058264960 |: [INFO]: 200:|7/10|2021-12-08-彰化縣-田中
2021-12-15 02:14:07,424 | 139657058264960 |: [INFO]: 200:|8/10|2021-12-08-澎湖縣-東吉島
2021-12-15 02:14:07,855 | 139657058264960 |: [INFO]: 200:|9/10|2021-12-08-澎湖縣-澎湖


,StationCode,StationName,StationCName,City,Date,ObsTime,StnPres,SeaPres,Temperature,Td_dew_point,RH,WS,WD,WSGust,WDGust,Precp,PrecpHour,SunShine,GloblRad,Visb,UVI,Cloud_Amount
0,466930,ZHUZIHU,竹子湖,臺北市,2021-12-08,1,953.9,1025.7,12.9,9.5,80,1.8,70,7.9,70,0,0.0,NA,0.0,NA,NA,NA
1,466930,ZHUZIHU,竹子湖,臺北市,2021-12-08,2,953.9,1025.6,13.0,9.8,81,2.1,70,7.2,10,0,0.0,NA,0.0,NA,NA,NA
2,466930,ZHUZIHU,竹子湖,臺北市,2021-12-08,3,953.4,1025.1,13.1,9.2,77,2.3,50,7.6,80,0,0.0,NA,0.0,NA,NA,NA
3,466930,ZHUZIHU,竹子湖,臺北市,2021-12-08,4,953.5,1025.2,13.1,10.3,83,1.9,60,8.5,90,0,0.0,NA,0.0,NA,NA,NA
4,466930,ZHUZIHU,竹子湖,臺北市,2021-12-08,5,953.7,1025.5,12.9,9.2,78,2.9,40,9.7,10,0,0.0,NA,0.0,NA,NA,NA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
235,467350,PENGHU,澎湖,澎湖縣,2021-12-08,20,1020.6,1021.9,20.1,14.7,71,5.7,10,14.2,20,0,0.0,NA,0.0,25.0,0,3.0
236,467350,PENGHU,澎湖,澎湖縣,2021-12-08,21,1020.7,1022.0,20.1,14.9,72,6.0,10,13.8,30,0,0.0,NA,0.0,25.0,0,3.0
237,467350,PENGHU,澎湖,澎湖縣,2021-12-08,22,1020.8,1022.1,20.1,14.9,72,5.3,10,16.4,10,0,0.0,NA,0.0,NA,0,NA
238,467350,PENGHU,澎湖,澎湖縣,2021-12-08,23,1020.9,1022.2,20.0,15.2,74,5.7,10,13.2,360,0,0.0,NA,0.0,NA,0,NA


# Threading 運用子程序爬蟲

In [ ]:
from threading import Thread

In [ ]:
target = pd.date_range('2021-12-01', '2021-12-11').to_pydatetime().tolist()
target_f =[Date.strftime("%Y-%m-%d") for Date in target]

In [ ]:
thread_list = []
for index,date in enumerate(target_f):
  thread = Thread(target=Codis,args=(date,match[0:12],))
  thread_list.append(thread)
  thread_list[index].start()
# show logging
for t in thread_list:
  t.join()

2021-12-15 02:14:25,295 | 139656308332288 |: [INFO]: 200:|0/12|2021-12-04-臺北市-竹子湖
2021-12-15 02:14:25,303 | 139656291546880 |: [INFO]: 200:|0/12|2021-12-06-臺北市-竹子湖
2021-12-15 02:14:25,309 | 139656249321216 |: [INFO]: 200:|0/12|2021-12-11-臺北市-竹子湖
2021-12-15 02:14:25,323 | 139656283154176 |: [INFO]: 200:|0/12|2021-12-07-臺北市-竹子湖
2021-12-15 02:14:25,324 | 139656257713920 |: [INFO]: 200:|0/12|2021-12-10-臺北市-竹子湖
2021-12-15 02:14:25,326 | 139656266106624 |: [INFO]: 200:|0/12|2021-12-09-臺北市-竹子湖
2021-12-15 02:14:25,326 | 139656274499328 |: [INFO]: 200:|0/12|2021-12-08-臺北市-竹子湖
2021-12-15 02:14:25,441 | 139656325117696 |: [INFO]: 200:|0/12|2021-12-02-臺北市-竹子湖
2021-12-15 02:14:25,452 | 139656333510400 |: [INFO]: 200:|0/12|2021-12-01-臺北市-竹子湖
2021-12-15 02:14:25,622 | 139656316724992 |: [INFO]: 200:|0/12|2021-12-03-臺北市-竹子湖
2021-12-15 02:14:25,623 | 139656299939584 |: [INFO]: 200:|0/12|2021-12-05-臺北市-竹子湖
2021-12-15 02:14:25,839 | 139656308332288 |: [INFO]: 200:|1/12|2021-12-04-基隆市-基隆
2021-12-15 02:14: